# How to build a simple Retriever LLM App with LangChain
* Very simple Retriever LLM App over a text data source. 
* Retriever Apps can answer questions about specific documents. 

In [1]:
#!pip install python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [3]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [4]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Install Chroma Database

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [6]:
#pip install langchain-chroma

## Documents
* A LangChain Document is intended to store text and metadata.
* Have 2 attributes:
    * `page_content`
    *  `metadata`

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.",
        metadata={"source": "us-presidents-doc"},
    ),
    Document(
        page_content="Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.",
        metadata={"source": "us-politics-doc"},
    ),
    Document(
        page_content="The Kennedy family is known for their significant influence in American politics and their extensive philanthropic efforts.",
        metadata={"source": "kennedy-family-doc"},
    ),
    Document(
        page_content="Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.",
        metadata={"source": "us-senators-doc"},
    ),
    Document(
        page_content="Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.",
        metadata={"source": "first-lady-doc"},
    ),
]

## Vector Stores vs. Retrievers
Let's break down the differences between **vector stores** and **retrievers** in a way that's easy to understand.

#### Vector Stores
Think of a vector store as a specialized storage space where information is kept in a very specific format:
- **Storing Vectors**: A vector store keeps information as vectors. These vectors are numerical representations of text, making it easier for machines to understand and compare information quickly.
- **Purpose**: The main goal of a vector store is to efficiently store and retrieve these vectors. When you need to find how similar two pieces of information are, the vector store helps by quickly comparing their vectors.
- **Usage**: They are crucial in systems where you need to perform similarity searches over large datasets. For example, finding documents that discuss similar topics or identifying similar user queries.

#### Retrievers
On the other hand, retrievers are more about actively finding information:
- **Retrieving Information**: A retriever takes a query (like a question or a search term) and looks through a database to find relevant information.
- **Purpose**: The purpose of a retriever is to sift through large amounts of data and bring back the most relevant documents or entries that answer the query.
- **Usage**: Retrievers are used in search engines, question-answering systems, and anywhere you need to pull out specific pieces of information from a large dataset quickly.

### Key Differences
- **Functionality**: Vector stores are focused on storing and retrieving numerical data representations, making them ideal for tasks that involve measuring similarity. Retrievers, meanwhile, are geared towards searching through text or data to find relevant information based on a query.
- **Output**: Vector stores return vectors or scores based on similarity measures, whereas retrievers provide a list of documents or data entries that are deemed relevant to the query.
- **Role in Systems**: Vector stores often serve as a backend component that supports the function of retrievers by providing the necessary data representations for comparison. Retrievers use this data to perform their role of finding and fetching relevant information.

In summary, vector stores and retrievers both help manage and utilize large data sets, but they do so in different ways. Vector stores focus on the storage and retrieval of data in a numerical format, while retrievers focus on retrieving relevant textual or data entries based on specific queries.

## Vector stores

We can use many vector stores in our LangChain applications. Here we will use a Chorma vector store.

In [4]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

#### similarity_search()
Imagine you have a large box full of various toys, and you're looking for toys that are similar to your favorite toy car. You might start by pulling out toys that are also cars, but then you narrow it down further to find cars that are the same color or size as your favorite one.

In computer terms, similarity search works similarly. It involves searching through a vast amount of data (like all those toys) to find items that are similar to a specific item you're interested in. This could be text, images, or any type of data.

When you use similarity search in a LangChain app here's how it typically goes:
1. **Representation**: converting words or sentences into numerical forms (called embeddings).
2. **Comparison**: Once everything is converted into numbers, compare these numbers to see how similar they are. This is like measuring the distance between two points.
3. **Retrieval**: The retriever then sorts these items by how similar they are to your query (what you're searching for) and shows you the results that are most similar.

The similarity_search() function returns documents based on similarity to a string query:

In [5]:
vectorstore.similarity_search("John")

[Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.'),
 Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.'),
 Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.'),
 Document(metadata={'source': 'first-lady-doc'}, page_content='Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.')]

#### similarity_search_with_score()
When we talk about `similarity_search_with_score` we're looking at a slightly more detailed process than just finding similar items. Here's how you can understand it:

1. **Input and Representation**: 
    - First, you have a query, which is what you're interested in finding similar items for. This could be a piece of text, like a question or a topic.
    - The system converts this query and all potential items that could be similar (like documents or pieces of text) into a numerical form that represents their meanings. This is usually done using models that produce embeddings.

2. **Scoring Similarities**:
    - Once everything is converted into these numerical embeddings, the system calculates the 'distance' between your query's embedding and the embeddings of other items. Closer distances mean they are more similar.
    - The system uses a similarity score to quantify how close or far each item is from your query. This score is typically between 0 and 1, where 1 means extremely similar and 0 means not similar at all.

3. **Ranking and Retrieval**:
    - Based on these scores, the system ranks all the items from most similar to least similar.
    - It then presents you with a list of items, each with a similarity score showing how closely it matches your query.

When using similarity search tools like the ones discussed in the LangChain course, these tools often convert text into numerical forms, or vectors, to measure how similar they are to each other. However, the way these vectors are stored and compared can differ depending on the tool or provider you use—each one might have its own method for scoring the similarities.

Unlike some other tools that give a similarity score where a higher number means more similar, Chroma does the opposite. It uses a distance metric for scoring. In this case:
- **A smaller distance means more similarity**: If the distance score is close to 0, it suggests that the items are very similar.
- **A larger distance means less similarity**: If the distance score is higher, it suggests that the items are quite different.

So, in simple terms, when you’re using Chroma's vector store for similarity search, remember that you’re looking for smaller numbers (or distances) to find more similar items, as these scores vary inversely with the similarity—smaller is better!

In [6]:
vectorstore.similarity_search_with_score("John")

[(Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.'),
  0.45017221570014954),
 (Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.'),
  0.46441298723220825),
 (Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.'),
  0.47508904337882996),
 (Document(metadata={'source': 'first-lady-doc'}, page_content='Jacqueline Kennedy Onassis, wife of John F. Kennedy, was an iconic First Lady known for her style, poise, and dedication to cultural and historical preservation.'),
  0.48099976778030396)]

## Retrievers

* We can create a retriever manually, but this is not the option we will use most frequently. Once we choose what method we wish to use to retrieve documents, we can create a retriever using RunnableLambda. The code below will build a retriever around the similarity_search method:

In [7]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=2)  # select top result

retriever.batch(["John", "Robert"])

[[Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.'),
  Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.')],
 [Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.'),
  Document(metadata={'source': 'us-senators-doc'}, page_content='Edward M. Kennedy, often known as Ted Kennedy, was a U.S. Senator who played a major role in American legislation over several decades.')]]

* Most of the times we will use the .as_retriever() function to create a Retriever using the vector store:

In [9]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["John", "Robert"])

[[Document(metadata={'source': 'us-presidents-doc'}, page_content='John F. Kennedy served as the 35th president of the United States from 1961 until his assassination in 1963.')],
 [Document(metadata={'source': 'us-politics-doc'}, page_content='Robert F. Kennedy was a key political figure and served as the U.S. Attorney General; he was also assassinated in 1968.')]]

#### Retrievers are runnables
LangChain VectorStore objects are not Runnables, and so they cannot immediately be integrated into LangChain Expression Language chains. On the contrary, LangChain Retrievers are Runnables.
* See how we use a retriever inside a LCEL chain:

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

chain = {
    "context": retriever, 
    "question": RunnablePassthrough()} | prompt | llm

In [11]:
response = chain.invoke("tell me about Jackie")

print(response.content)

Jackie was the wife of John F. Kennedy and was known for her style, poise, and dedication to cultural and historical preservation.


## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 004-invoke-stream-batch.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-retriever-app.py